In [29]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
from WhatIfAnalysis import GoalSeek
# import sectionproperties as shape

from ASCE import load_combos

from steel_design.AISC import aisc
steel_density = 490 / 12**3 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

from wood_design.sawn_lumber.ref_design_values.table_4a import table_4a_load
from wood_design.sawn_lumber.ref_design_values.adjustment_factors import SawnLumberAdjustmentFactors
from wood_design.sawn_lumber.ref_design_values.C_F import C_F_load
table_4a = table_4a_load(species="DF")

<div style="page-break-after: always;"></div>

# Calculations

## Compression

In [30]:
P = 1000 #lbs

$$F_{cE} = \frac{\pi^2 E}{(l / r)^2}$$

$$r = \sqrt{\frac{I}{A}}$$

$$C_P = \frac{1 + \frac{F_{cE}}{F_{c*}}}{2c} - \sqrt{\left(\frac{1 + \frac{F_{cE}}{F_{c*}}}{2c}\right)^2 - \frac{F_{cE}}{cF_{c*}}}$$

In [31]:
table_4a_load("DF")

,Fb,Ft,Fv,Fc_perp,Fc,E,Emin,G
Grade,,,,,,,,
Select Structural,1500,1000,180,625,1700,1900000,690000,0.5
No. 1 & Btr,1200,800,180,625,1550,1800000,660000,0.5
No. 1,1000,675,180,625,1500,1700000,620000,0.5
No. 2,900,575,180,625,1350,1600000,580000,0.5
No. 3,525,325,180,625,775,1400000,510000,0.5


In [32]:
C_D = 1.0
C_M = 1.0
C_t = 1.0
C_F = 1.0
C_i = 1.0

In [33]:
Ep_min = 69000 #psi
l_e = 72 #in
d = 5.5 #in
b = 5.5 #in
d_hole = 2 #in

# Square section
I_square = (b * d**3) / 12  # in^4
A_square = b * d  # in^2

# Circular hole
I_circle = (np.pi * d_hole**4) / 64  # in^4
A_circle = np.pi * (d_hole/2)**2  # in^2

# Square section with hole
I = I_square - I_circle  # in^4
A = A_square - A_circle  # in^2

r = sqrt(I / A) #in
Fc = 1350 #psi
Fb = 900 #psi
c = 0.80 #sawn lumber


In [34]:
FcE = pi**2 * Ep_min / (l_e/r)**2
Fc_st = Fc * C_D * C_M * C_t * C_F * C_i
C_P = (1 + (FcE/Fc_st))/(2*c) - np.sqrt(((1 + (FcE/Fc_st))/(2*c))**2 - (FcE/(c*Fc_st)))
print(f"F_cE = {round(FcE, 3)}")
print(f"C_P = {round(C_P, 3)}")

F_cE = 365.724
C_P = 0.254


In [35]:
Fc_pr = Fc_st * C_P
f"F_c' = {round(Fc_pr, 2)} psi"

"F_c' = 342.45 psi"

In [36]:
P_a = Fc_pr * A #lbs
f"P_a = {round(P_a, 1)} lbs"

'P_a = 9283.2 lbs'

## Tension

In [37]:
M = 0 #lb-in

In [38]:
C_fu = 1.0
C_r = 1.0

In [39]:
R_B = sqrt(l_e * d / b**2)
FbE = 1.20 * Ep_min / R_B**2 #psf
Fb_st = Fb * C_D * C_M * C_t * C_F * C_i * C_r #psi
C_L = (1 + (FbE/Fb_st))/(2*c) - np.sqrt(((1 + (FbE/Fb_st))/(2*c))**2 - (FbE/(c*Fb_st)))
f"C_L = {round(C_L, 3)}"

'C_L = 0.969'

In [41]:
Fb_pr = Fb_st * C_L #psi
f"Fb_pr = {round(Fb_pr, 2)} psi"

'Fb_pr = 872.1 psi'

In [42]:
# Square section
S_square = I_square / (d/2)  # in^3

# Circular hole
S_circle = I_circle / (d_hole/2)  # in^3

# Square section with hole
S = S_square - S_circle  # in^3

M_a = Fb_st * S
f"M_a = {round(M_a, 2)} lb-in"

'M_a = 24249.39 lb-in'